## **Problem Description**:

There are structural multi-parametric MRI (mpMRI) scans for different subjects, in DICOM format. The exact mpMRI scans included are:

* Fluid Attenuated Inversion Recovery (FLAIR)
* T1-weighted pre-contrast (T1w)
* T1-weighted post-contrast (T1Gd)
* T2-weighted (T2)

`train_labels.csv` - file contains the target MGMT_value for each subject in the training data **(e.g. the presence of MGMT promoter methylation)**.

So, it's a binary classification problem.

## **A simple solution**:

* For each patient, we consider 4 sequences (FLAIR, T1w, T1Gd, T2), and for each of those sequences take a slice randomly. Idea from [https://github.com/zabir-nabil/Fibro-CoSANet](https://github.com/zabir-nabil/Fibro-CoSANet)

* Construct a 4-channel image out of these 4 sequences.

* Design a 4 channel pytorch model.

* Perform binary classification.

### **Updates** v3:

* Added augmentation. Check out the augmentation notebook: https://www.kaggle.com/furcifer/mri-data-augmentation-pipeline
* Added few heuristics to avoid black/empty scans.
* Added full training script with weights saving.
* Added modified efficient-net (4 channels) as model.

### **Updates** v4:

* Experimenting with first training without any augmentation, and after few epochs adding augmentation.
* Training longer

## **Check out my other kernels**

### ⚡ **Training kernel:** https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train/

### ⚡ **Inference kernel:** https://www.kaggle.com/furcifer/torch-effnet3d-for-mri-no-inference/



## **CNN with 4 channels**

In [ ]:
from IPython.display import Image
Image("../input/diagram/diagram.png")

In [ ]:
import os
import glob
from tqdm import tqdm_notebook as tqdm
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
sys.path.append('../input/efficientnet-pytorch')
sys.path.append('../input/efficientnet/EfficientNet-PyTorch-master/')

In [ ]:
path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'
train_data = pd.read_csv(os.path.join(path, 'train_labels.csv'))
print('Num of train samples:', len(train_data))

In [ ]:
train_data.head()

## **Augmentation**

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.1, aug)

seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.5), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.05),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.LinearContrast((0.5, 2.0))
                    )
                ]),
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

In [ ]:
def dicom2array(paths, voi_lut=True, fix_monochrome=True, remove_black_boundary=True, aug = False):
    
    for path in paths:
        dicom = pydicom.read_file(path)
        # VOI LUT (if available by DICOM device) is used to
        # transform raw DICOM data to "human-friendly" view
        if voi_lut:
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
        if data.max() > 0.0: # avoiding black images (if possible)
            break
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    if remove_black_boundary: # we get slightly more details
        (x, y) = np.where(data > 0)
        if len(x) > 0 and len(y) > 0:
            x_mn = np.min(x)
            x_mx = np.max(x)
            y_mn = np.min(y)
            y_mx = np.max(y)
            if (x_mx - x_mn) > 10 and (y_mx - y_mn) > 10:
                data = data[:,np.min(y):np.max(y)]
    data = cv2.resize(data, (512, 512))
    if aug and random.randint(0,1) == 1: # augmenting only 50% of the time
        data = seq(images=data)
    return data

def load_rand_dicom_images(scan_id, split = "train", aug = False):
    """
    send 4 random slices of each modality
    """
    if split != "train" or split != "test":
        split = "train"
    flair = sorted(glob.glob(f"{path}/{split}/{scan_id}/FLAIR/*.dcm"))
    flair_img = dicom2array(random.sample(flair, max(len(flair)//2, 1)), aug = aug)
    t1w = sorted(glob.glob(f"{path}/{split}/{scan_id}/T1w/*.dcm"))
    t1w_img = dicom2array(random.sample(t1w, max(len(t1w)//2, 1)), aug = aug)
    t1wce = sorted(glob.glob(f"{path}/{split}/{scan_id}/T1wCE/*.dcm"))
    t1wce_img = dicom2array(random.sample(t1wce, max(len(t1wce)//2, 1)), aug = aug)
    t2w = sorted(glob.glob(f"{path}/{split}/{scan_id}/T2w/*.dcm"))
    t2w_img = dicom2array(random.sample(t2w, max(len(t2w)//2, 1)), aug = aug)
    
    return np.array((flair_img, t1w_img, t1wce_img, t2w_img)).T

In [ ]:
load_rand_dicom_images("00000", aug = True).shape

In [ ]:
def plot_imgs(imgs, cols=4, size=7, is_rgb=True, title="", cmap='gray', img_size=(512,512)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i in range(4):
        img = imgs[:,:,i]
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:
slices = load_rand_dicom_images("00000")
plot_imgs(slices)
aug_slices = load_rand_dicom_images("00000", aug = True)
plot_imgs(aug_slices)

In [ ]:
# let's write a simple pytorch dataloader


class BrainTumor(Dataset):
    def __init__(self, path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification', split = "train", validation_split = 0.2):
        # labels
        train_data = pd.read_csv(os.path.join(path, 'train_labels.csv'))
        self.labels = {}
        brats = list(train_data["BraTS21ID"])
        mgmt = list(train_data["MGMT_value"])
        for b, m in zip(brats, mgmt):
            self.labels[str(b).zfill(5)] = m
            
        if split == "valid":
            self.split = split
            self.ids = [a.split("/")[-1] for a in sorted(glob.glob(path + f"/train/" + "/*"))]
            self.ids = self.ids[:int(len(self.ids)* validation_split)] # first 20% as validation
        elif split == "train":
            self.split = split
            self.ids = [a.split("/")[-1] for a in sorted(glob.glob(path + f"/{split}/" + "/*"))]
            self.ids = self.ids[int(len(self.ids)* validation_split):] # last 80% as train
        else:
            self.split = split
            self.ids = [a.split("/")[-1] for a in sorted(glob.glob(path + f"/{split}/" + "/*"))]
            
    
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        imgs = load_rand_dicom_images(self.ids[idx], self.split, aug = True)
        
        transform = transforms.Compose([transforms.ToTensor()]) # transforms.Normalize((0.5, 0.5, 0.5, 0.5), (0.5, 0.5, 0.5, 0.5))
        imgs = transform(imgs)
        
        imgs = imgs - imgs.min()
        imgs = (imgs + 1e-5) / (imgs.max() - imgs.min() + 1e-5)
        
        if self.split != "test":
            label = self.labels[self.ids[idx]]
            return torch.tensor(imgs, dtype = torch.float32), torch.tensor(label, dtype = torch.long)
        else:
            return torch.tensor(imgs, dtype = torch.float32)

In [ ]:
# testing the dataloader
# testing the dataloader
train_bs = 10
val_bs = 2

train_dataset = BrainTumor()
train_loader = DataLoader(train_dataset, batch_size=train_bs, shuffle=True, num_workers = 8)

val_dataset = BrainTumor(split="valid")
val_loader = DataLoader(val_dataset, batch_size=val_bs, shuffle=True, num_workers = 8)

In [ ]:
for img, label in train_loader:
    print(img.shape)
    print(img.min())
    print(img.mean())
    print(img.max())
    print(label.shape)
    break

for img, label in val_loader:
    print(img.shape)
    print(label.shape)
    break

## **CNN (dummy) + EfficientNet**

In [ ]:
# let's write our simplest cnn

class SimpleCNN(nn.Module): 
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=9) # 4 incoming channels
        self.conv2 = nn.Conv2d(64, 32, kernel_size=7)
        self.conv2_drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(32, 16, kernel_size=5)
        self.conv3_drop = nn.Dropout2d()
        self.conv4 = nn.Conv2d(16, 8, kernel_size=3)
        self.conv4_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(200, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 4))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 4))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [ ]:
from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch.utils import Conv2dStaticSamePadding

PATH = "../input/efficientnet-pytorch/efficientnet-b1-dbc7070a.pth"
model = EfficientNet.from_name('efficientnet-b1')
model.load_state_dict(torch.load(PATH))

# augment model with 4 channels

model._conv_stem = Conv2dStaticSamePadding(4, 32, kernel_size = (3,3), stride = (2,2), 
                                                             bias = False, image_size = 512)

model._fc = torch.nn.Linear(in_features=1280, out_features=2, bias=True)

# https://github.com/zabir-nabil/Fibro-CoSANet    # will update later        

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr = 0.001, weight_decay=0.02)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
n_epochs = 2

In [ ]:
print(model)
model(torch.randn(1, 4, 512, 512))

## **Training**

In [ ]:
# helper
def one_hot(arr):
    return [[1, 0] if a_i == 0 else [0, 1] for a_i in arr]

In [ ]:
# let's train
gpu = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
model.to(gpu)

train_loss = []
val_loss = []
train_roc = []
val_roc = []
best_roc = 0.0

for epoch in range(n_epochs):  # loop over the dataset multiple times
    y_all = []
    outputs_all = []
    running_loss = 0.0
    roc = 0.0
    
    model.train()
    for i, data in tqdm(enumerate(train_loader, 0)):
        x, y = data
        
        # x = torch.unsqueeze(x, dim = 1)
        x = x.to(gpu)
        y = y.to(gpu)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        y_all.extend(y.tolist())
        outputs_all.extend(outputs.tolist())
    
    roc += roc_auc_score(one_hot(y_all), outputs_all) / train_bs
    print(f"epoch {epoch+1} train loss: {running_loss} train roc: {roc}")
    
    train_loss.append(running_loss)
    train_roc.append(roc)

    y_all = []
    outputs_all = []
    running_loss = 0.0
    roc = 0.0 
    
    model.eval()
    for i, data in tqdm(enumerate(val_loader, 0)):

        x, y = data
        
        # x = torch.unsqueeze(x, dim = 1)
        x = x.to(gpu)
        y = y.to(gpu)

        # forward
        outputs = model(x)
        loss = criterion(outputs, y)

        # print statistics
        running_loss += loss.item()
        y_all.extend(y.tolist())
        outputs_all.extend(outputs.tolist())
    
    roc += roc_auc_score(one_hot(y_all), outputs_all) / val_bs
    scheduler.step(running_loss)
        
    print(f"epoch {epoch+1} val loss: {running_loss} val roc: {roc}")
    
    val_loss.append(running_loss)
    val_roc.append(roc)
    
    if roc > best_roc:
        best_roc = roc
        torch.save(model.state_dict(), f'best_roc_{round(roc, 2)}_loss_{round(running_loss, 2)}.pt')

In [ ]:
plt.plot(train_loss, label = 'train loss')
plt.plot(val_loss, label = 'val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train loss', 'val loss'])
plt.show()

plt.plot(train_roc, label = 'train roc')
plt.plot(val_roc, label = 'val roc')
plt.xlabel('epochs')
plt.ylabel('roc auc')
plt.legend(['train roc', 'val roc'])
plt.show()

In [ ]:
submission = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
submission.to_csv("submission.csv", index=False)
submission